In [1]:
import os
os.environ['AOC_SESSION'] = '53616c7465645f5f37c74a4bd1a2394b4fa5dcbb449244bb183873f839c409c1c2b5d3191175e6de6040979aeb78f7894665a168392bc55c2d58d905f027b397'

In [2]:
import aocd
from aocd.models import Puzzle
day = 21
year = 2024
puzzle = Puzzle(year=year, day=day)
# data = aocd.get_data(day=day, year=year)
with open('./data/input_{:02d}'.format(day), 'w') as fh:
    fh.write(puzzle.input_data)

In [3]:
from functools import cache
from itertools import permutations, product
from heapq import heappop, heappush

In [4]:
test_data = """029A
980A
179A
456A
379A"""
data_test = test_data.splitlines()

In [5]:
data = puzzle.input_data.splitlines()
len(data), data[:10]

(5, ['964A', '140A', '413A', '670A', '593A'])

In [6]:
def mdist(pos0, pos1):
    if isinstance(pos1, complex):
        return int(abs(pos1.real - pos0.real) + abs(pos1.imag - pos0.imag))
    return int(abs(pos1[0] - pos0.real) + abs(pos1[1] - pos0.imag))

def get_dir(cpl):
    return {
        -1: '<',
        1: '>',
        1j: 'v',
        -1j: '^'
    }[cpl]

nbrs = [-1, 1, -1j, 1j]

@cache
def get_path_numeric(_from, _to):
    # get all direct paths from numbers to numbers
    position = {
        'A': 2+3j,
        '7': 0+0j,
        '8': 1+0j,
        '9': 2+0j,
        '4': 0+1j,
        '5': 1+1j,
        '6': 2+1j,
        '1': 0+2j,
        '2': 1+2j,
        '3': 2+2j,
        '0': 1+3j,
    }
    possible = set(position.values())
    # print(possible)
    delta = position[_to] - position[_from]
    paths = []
    stack = [(position[_from], '')]
    while len(stack):
        pos, path = stack.pop()
        # print(pos, path)
        if pos == position[_to]:
            paths.append(path)
        for nbr in nbrs:
            npos = pos + nbr
            if npos not in possible:
                continue
            if mdist(npos, position[_to]) < mdist(pos, position[_to]):
                npath = path + get_dir(nbr)
                stack.append((npos, npath))
    return paths

@cache
def get_path_directional(_from, _to):
    # get all direct paths for directional pad
    position = {
        'A': 2+0j,
        '>': 2+1j,
        '<': 0+1j,
        'v': 1+1j,
        '^': 1+0j,
    }
    possible = set(position.values())
    # print(possible)
    delta = position[_to] - position[_from]
    paths = []
    stack = [(position[_from], '')]
    while len(stack):
        pos, path = stack.pop()
        # print(pos, path)
        if pos == position[_to]:
            paths.append(path)
        for nbr in nbrs:
            npos = pos + nbr
            if npos not in possible:
                continue
            if mdist(npos, position[_to]) < mdist(pos, position[_to]):
                npath = path + get_dir(nbr)
                stack.append((npos, npath))
    return paths

@cache
def replace_get_all(path):
    """get all directional paths that would create the input"""
    start = 'A'
    paths = []
    for char in path:
        end = char
        paths.append([p + 'A' for p in get_path_directional(start, end)])
        start = end
        
    return ["".join(p) for p in product(*paths)]


def get_lvl0_paths(code):
    """get paths on numeric keypad that create the given code"""
    start = 'A'
    paths = ['']
    for char in code:
        end = char
        paths = [path_start + path_add + 'A' for path_add in get_path_numeric(start, end) for path_start in paths]
        start = end
    return paths

            
def solve(code, maxlvl=2):
    """start recursive search with paths on numeric pad"""
    paths = get_lvl0_paths(code)
    return min([solve_directional(path, 0, maxlvl) for path in paths])

@cache
def solve_directional(path, lvl, maxlvl=2):
    """
    - split path at A
    - then look recursively for shortest directional path that creates the input
    - return sum of minimums for each part
    - at max level return length of path
    """
    # print(path, lvl)
    if lvl == maxlvl:
        return len(path)
    
    parts = path.split('A')
    
    ans = 0
    for i, part in enumerate(parts):
        # ans += (solve_directional(replace(part+'A' if i < (len(parts)-1) else ''), lvl+1, maxlvl=maxlvl))
        ans += min([solve_directional(p, lvl+1, maxlvl=maxlvl) for p in replace_get_all(part+'A' if i < (len(parts)-1) else '')])
    return ans
    

In [7]:
%%time
d = data_test
d = data

res = 0
for code in d:
    seq = solve(code, maxlvl=2)
    print(code, seq)
    res += seq * int(code[:3])
print(res)

964A 72
140A 70
413A 70
670A 68
593A 74
197560
CPU times: user 1.84 ms, sys: 187 µs, total: 2.03 ms
Wall time: 2.01 ms


In [88]:
puzzle.answer_a = res

That's the right answer!  You are one gold star closer to finding the Chief Historian. [Continue to Part Two]


In [9]:
# Part 2

In [10]:
%%time
# d = data_test
d = data

res = 0
for code in d:
    seq_len = solve(code, maxlvl=25)
    print(code, seq_len)
    res += seq_len * int(code[:3])
print(res)

964A 85006969638
140A 87513499934
413A 87288844796
670A 84248089344
593A 93831469524
242337182910752
CPU times: user 8.72 ms, sys: 43 µs, total: 8.76 ms
Wall time: 8.63 ms


In [11]:
res

242337182910752

In [322]:
puzzle.answer_b = res

That's the right answer!  You are one gold star closer to finding the Chief Historian.You have completed Day 21! You can [Shareon
  Bluesky
Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
